In [95]:
/*
Example of Pricing an American Option on a Dividend Paying Stock in QuantLib in Java
See https://bnikolic.co.uk/blog/ql-american-disc-dividend and QuantLib-SWIG Java Equity Option examples. 
Run this on in Jupyter using IJava (see https://bnikolic.co.uk/blog/java/c++/jupyter/2023/04/20/ijava-binder)
*/

In [96]:
%jars "QuantLib.jar"

In [97]:
import org.quantlib.DayCounter;
import org.quantlib.Actual365Fixed;

import org.quantlib.Month;
import org.quantlib.Date;
import org.quantlib.TARGET;
import org.quantlib.Calendar;
import org.quantlib.Period;


import org.quantlib.Exercise;
import org.quantlib.AmericanExercise;
import org.quantlib.Option;
import org.quantlib.Payoff;
import org.quantlib.PlainVanillaPayoff;
import org.quantlib.DividendVanillaOption;

import org.quantlib.DateVector;
import org.quantlib.DoubleVector;
import org.quantlib.TimeUnit;

import org.quantlib.QuoteHandle;
import org.quantlib.SimpleQuote;

import org.quantlib.YieldTermStructureHandle;
import org.quantlib.FlatForward;
import org.quantlib.BlackVolTermStructureHandle;
import org.quantlib.BlackConstantVol;
import org.quantlib.BlackScholesProcess;

import org.quantlib.FdBlackScholesVanillaEngine;

import org.quantlib.Settings;

In [98]:
Calendar calendar = new TARGET();
DayCounter dayCounter=new Actual365Fixed();

Date settlementDate = new Date(17, Month.May, 2023);
Date maturity = new Date(17, Month.May, 2024);

Settings.instance().setEvaluationDate(settlementDate);


double strike = 100;
double underlying = 100;
double riskFreeRate = 0.04;
double volatility = 0.20;

In [99]:
Exercise americanExercise = new AmericanExercise(settlementDate,
                                                maturity);
                                                
PlainVanillaPayoff payoff = new PlainVanillaPayoff(Option.Type.Call, strike);                                                

In [100]:
DateVector divDates = new DateVector();
DoubleVector divAmounts = new DoubleVector();

Date divRoot = new Date(1, Month.March, 2023);
for (int i = 1; i <= 4; i++) {
        Date d = divRoot.add(new Period(3*i, TimeUnit.Months));
           divDates.add(d);
           divAmounts.add(1.0);
        }
        
DividendVanillaOption opt = new DividendVanillaOption(payoff,
                            americanExercise,
                            divDates,
                            divAmounts);
        

QuoteHandle underlyingH = new QuoteHandle(new SimpleQuote(underlying));

YieldTermStructureHandle flatTermStructure =
            new YieldTermStructureHandle(new FlatForward(
                                  settlementDate, riskFreeRate, dayCounter));
    
BlackVolTermStructureHandle flatVolatility =
        new BlackVolTermStructureHandle(new BlackConstantVol(
                          settlementDate, calendar, volatility, dayCounter));

BlackScholesProcess stochasticProcess =
            new BlackScholesProcess(underlyingH,
                                      flatTermStructure,
                                          flatVolatility);

opt.setPricingEngine( new FdBlackScholesVanillaEngine(stochasticProcess) );


In [101]:
opt.NPV();

7.815956882111247